In [1]:
!pip install transformers[sentencepiece]

In [2]:
import re
import requests
import pandas as pd
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import time
import torch

Подсоединение к Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
initTime = time.time()

model_checkpoint = "ai-forever/RuM2M100-1.2B"
model = M2M100ForConditionalGeneration.from_pretrained(model_checkpoint)
tokenizer = M2M100Tokenizer.from_pretrained(model_checkpoint, src_lang="ru", tgt_lang="ru")

print ("Time initialization: ", time.time() - initTime)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.49G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/465k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/489k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

Time initialization:  53.868499994277954


In [5]:
with open('/content/drive/My Drive/errorsents.txt', encoding="utf-8") as f:
  sentmarked_list = f.readlines()

Собираем информацию о словах с ошибками

In [6]:
errPattern = re.compile("_([\w\s-]+)_")
sent_list = []
errtup_list = []
for s in sentmarked_list:
    errIter = errPattern.finditer(s)
    ss = re.sub("_", "", s)
    sent_list.append(ss)
    j = 0
    errtup = []
    for err in errIter:
        errtup.append((err.start()-j*2, err.end()-2*(1 + j)))
        j += 1
    errtup_list.append(errtup)

Загружаем предложения без ошибок.
В последствии здесь тоже могут выявлены ошибки, поэтому также
формируем errors_pos

In [7]:
with open('/content/drive/My Drive/correctsents.txt', encoding="utf-8") as f:
  sentcorrect_list = f.readlines()

In [8]:
for s in sentcorrect_list:
    errIter = errPattern.finditer(s)
    ss = re.sub("_", "", s)
    sent_list.append(ss)
    j = 0
    errtup = []
    for err in errIter:
        errtup.append((err.start()-j*2, err.end()-2*(1 + j)))
        j += 1
    errtup_list.append(errtup)

Собираем DataFrame

In [9]:
sentences = pd.DataFrame(list(zip(sent_list, errtup_list)), columns =['sentences', 'errors_pos'])

Начальные параметры

In [10]:
p = re.compile(r'[\w-]+')

TP = 0
TN = 0
FP = 0
FN = 0

Levenshtein distance

In [11]:
def levenshtein_distance(s, t):
    m, n = len(s), len(t)
    if m < n:
        s, t = t, s
        m, n = n, m
    d = [list(range(n + 1))] + [[i] + [0] * n for i in range(1, m + 1)]
    for j in range(1, n + 1):
        for i in range(1, m + 1):
            if s[i - 1] == t[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                d[i][j] = min(d[i - 1][j], d[i][j - 1], d[i - 1][j - 1]) + 1
    return d[m][n]

Word similarity

In [12]:
def compute_similarity(input_string, reference_string):
    distance = levenshtein_distance(input_string, reference_string)
    max_length = max(len(input_string), len(reference_string))
    similarity = 1 - (distance / max_length)
    return similarity

In [30]:
def m2m100_sent(text, model, tokenizer):
  listErr = []
  errw = []
  encodings = tokenizer(text, return_tensors="pt")
  generated_tokens = model.generate(**encodings, forced_bos_token_id=tokenizer.get_lang_id("ru"))
  answer = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
  sw_list = text.split()
  aw_list = answer[0].split()
  offset = 0
  merged = 0
  print (sw_list)
  print (aw_list)
  # Проходим по всем словам в исходном предложении
  for nw in range(len(sw_list)):
    cs = compute_similarity(sw_list[nw], aw_list[nw+offset])
    print (cs, sw_list[nw])
    # Если merged = -1 (то есть правильное слово слито, то пропускаем цикл)
    if offset > -1:
      # Если похожесть между 1 и 0.8, то считаем, что это ошибка в одном слове (сдвиг = 0)
      if cs < 1 and cs >=0.8:
          errw.append((nw, 0))
      # Если похожесть меньше либо равна 0.6, отрабатываем слияние и разделение слов
      elif cs <=0.6:
          # Если исходное слово меньше, то слияние
          if len(sw_list) < len(aw_list):
            try:
              cs_space = compute_similarity(sw_list[nw], aw_list[nw+offset]+aw_list[nw+offset+1])
              # При слиянии- разделении (дополненные) слова должны соответствовать друг другу
              if cs_space <= 1 and cs_space >=0.8:
                errw.append((nw, 1))
                offset += 1
              else:
                print(f'ERROR_SPACE: Word number: {nw}, input word: {sw_list[nw]}, output word: {aw_list[nw+offset]}, compute_similarity: {cs}')
            except:
              print(f'EXCEPTION_1: Word number: {nw}, input word: {sw_list[nw]}, output word: {aw_list[nw+offset]}, compute_similarity: {cs}')
          # Если исходное слово больше, то разделение
          elif len(sw_list) > len(aw_list):
            try:
              cs_merge = compute_similarity(sw_list[nw]+sw_list[nw+1], aw_list[nw+offset])
              # При слиянии- разделении (дополненные) слова должны соответствовать друг другу
              if cs_merge <= 1 and cs_merge >=0.8:
                errw.append((nw, -1))
                merged = -1
                offset -= 1
              else:
                print(f'ERROR_MERGE: Word number: {nw}, input word: {sw_list[nw]}, output word: {aw_list[nw+offset]}, compute_similarity: {cs}')
            except:
                print(f'EXCEPTION_2: Word number: {nw}, input word: {sw_list[nw]}, output word: {aw_list[nw+offset]}, compute_similarity: {cs}')
          else:
            print(f'ERROR_1: Word number: {nw}, input word: {sw_list[nw]}, output word: {aw_list[nw+offset]}, compute_similarity: {cs}')
      elif cs < 0.8 and cs >=0.6:
        print(f'ERROR_2: Word number: {nw}, input word: {sw_list[nw]}, output word: {aw_list[nw+offset]}, compute_similarity: {cs}')
    else:
      merged = 0
    print (errw)
    s_iter = p.finditer(text)
    for idx, word in enumerate(s_iter):
      for e in errw:
        if idx == e[0]:
          errorrDesc = {
             "word": word.group(),
             "start": word.start(),
             "end": word.end(),
          }
          listErr.append(errorrDesc)
    return listErr

Перебор всех предложений с DataFrame и сбор сведений для расчета метрики

In [31]:
outerrors = []
startPos = 0
for ind in sentences.index:
    if (ind % 10) == 0:
        print ("Index: ", ind, "   Time:", time.time() - initTime)
    if (ind == 3):
      break
    response = m2m100_sent(sentences["sentences"][ind], model, tokenizer)
    print (response)
    if len(response) == 0:
        # No errors found
        ntoken = len(p.findall(sentences["sentences"][ind]))
        nerr = len(sentences["errors_pos"][ind])
        # True Negative = number of words without errors
        TN += ntoken - nerr
        # False Negative - nerr errors were not found
        FN += nerr
        article_err = sentences["sentences"][ind]
    else:
        # errors were found: iter by found errors
        nfound = 0
        startPos = 0
        article_err = ""
        for e in response:
            article_err += sentences["sentences"][ind][startPos:e['start']] + "<mark>" + sentences["sentences"][ind][e['start']:e['end']] + "</mark>"
            startPos = e['end']
            # found errors correct or not
            iscorrect = 0
            for trueerr in sentences["errors_pos"][ind]:
                if trueerr[0] == e["start"]:
                    iscorrect = 1
                    nfound += 1
                    break
            if iscorrect == 1:
                # True Positive - error word was found
                TP += 1
        article_err += sentences["sentences"][ind][startPos:]
        # False Positive - wrong errors were found
        if (len(response) - nfound) > 0:
            FP += len(response) - nfound
        # False Negative - errors were not found
        if (len(sentences["errors_pos"][ind]) - nfound) > 0:
            FN += len(sentences["errors_pos"][ind]) - nfound
    outerrors.append(article_err)

Index:  0    Time: 3004.622858762741
['Точные', 'данные', 'по', 'поврежденном', 'имуществу', 'станут', 'известны', 'после', 'поквартирного', 'обхода.']
['Точные', 'данные', 'по', 'поврежденному', 'имуществу', 'станут', 'известны', 'после', 'поквартирного', 'обхода.']
1.0 Точные
[]
[]
['Сегодня', 'наши', 'войны', 'могли', 'помолиться', 'и', 'причастится', 'перед', 'святым', 'образом.']
['Сегодня', 'наши', 'воины', 'могли', 'помолиться', 'и', 'причастится', 'перед', 'святым', 'образом.']
1.0 Сегодня
[]
[]
['По', 'ее', 'словам,', 'на', 'данный', 'момент', 'единственный', 'средством', 'борьбы', 'против', 'нейросетей', 'в', 'этом', 'случае', 'должен', 'стать', 'человеческий', 'ум.']
['По', 'ее', 'словам,', 'на', 'данный', 'момент', 'единственный', 'средством', 'борьбы', 'против', 'нейросетей', 'в', 'этом', 'случае', 'должен', 'стать', 'человеческий', 'ум.']
1.0 По
[]
[]


Сохранение файла с метками

In [21]:
with open('/content/drive/My Drive/errorsmarks.txt', "w", encoding="utf-8") as file:
  file.writelines(outerrors)

Расчет метрики

In [22]:
print (f'TP: {TP}, FP: {FP}, FN: {FN} ')
precision = TP / (TP + FP)
recall = TP / (TP + FN)
fmeasure = 2 * recall * precision / (recall + precision)
print ("Precision: ", precision)
print ("Recall: ", recall)
print ("F-measure: ", fmeasure)

Precision:  1.0
Recall:  0.05
F-measure:  0.09523809523809523
